In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import *

In [3]:
import torch
from torch import nn

In [4]:
dim = 50

In [5]:
embeddings = load_glove_embeddings(dim)
len(embeddings)

400002

In [6]:
embeddings['wax']

tensor([ 0.1392,  0.7532, -0.1715, -0.6622, -0.1074,  0.3893, -0.4517, -0.7152,
        -0.3411,  0.9171,  0.5661,  0.2525,  1.0873,  0.9969,  0.1705,  0.8662,
         0.0614, -0.1375, -0.2858, -1.2783,  0.6066, -0.3275,  0.7814, -0.7399,
         0.1129, -0.3309, -1.3136,  1.4563,  0.0447, -1.1595,  0.3302, -0.5312,
        -0.6305,  0.0097, -0.5013, -0.0992,  0.1624,  0.9057,  0.7850, -0.1771,
         0.7864, -0.1800,  0.1099, -0.1697,  0.5408,  0.3937,  0.2866, -0.6887,
        -0.3778, -1.0411])

In [7]:
data = [
    ("sequencer", "dna sequencing was carried out using the dye termination kit and an automatic sequencer", "an apparatus for determining the sequence of amino acids or other monomers in a biological polymer"),
    ("order", "the templars were also known as the order of christ", "a society of knights bound by a common rule of life and having a combined military and monastic character"),
    ("horde", "an army or tribe of nomadic warriors", "the viking hordes returned to york this weekend as fierce armoured warriors mingled with the city centre crowds ."),
    ("anaemic", "although it has been thought of as a symptom of iron deficiency , it is more commonly discovered in patients who are not anemic .", "suffering from anaemia")
]

In [8]:
endcoder = init_encoder()
decoder = init_decoder()
data_loader = init_data_loader()

In [ ]:
epochs = 100
criterion = nn.NLLLoss()
encoder_optim = torch.optim.Adam()
decoder_optim = torch.optim.Adam()

for i in range(epochs):
    epoch_loss = 0
    for x, y in data_loader:
        contexts = encoder.forward(x)
        outputs = decoder.forward(contexts)
        
        loss = criterion(outputs, y)
        loss.backward()
        
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        epoch_loss += loss.item() / len(x)
        
    print(f'epoch_loss={epoch_loss})

In [63]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

In [54]:
sents = sorted([d[1].split() for d in data], key=lambda s: len(s), reverse=True)

In [56]:
sents_emb = [embeddings.sentence_to_tensor(sent) for sent in sents]

In [57]:
lens = [len(s) for s in sents_emb]
max_len = max(lens)

In [59]:
batch = pack_padded_sequence(pad_sequence(sents_emb), lens)

In [70]:
lstm = nn.LSTM(dim, dim, bidirectional=False)

In [80]:
out, hid = lstm(batch)

In [20]:
out

tensor([[[ 5.3968e-02, -1.0389e-02,  3.4268e-02,  6.9217e-02,  1.8372e-01,
          -4.1212e-02, -1.0714e-01,  2.5864e-02, -1.5515e-01,  4.8334e-02,
           1.1818e-01, -1.6983e-01, -1.4686e-01, -4.2101e-02,  4.8452e-02,
           2.1894e-02, -1.4428e-01,  1.5644e-01, -5.9404e-02, -9.5048e-02,
           1.5690e-01,  1.4675e-01, -4.0267e-02,  1.8027e-01, -2.0773e-03,
           1.0477e-01, -6.1649e-02, -1.9465e-02,  7.0539e-02,  2.8197e-02,
           5.4142e-02,  6.5958e-02,  1.3296e-01, -2.8208e-02, -3.3209e-02,
          -1.4955e-01, -7.1683e-02,  6.0909e-02, -6.2276e-02,  2.3516e-02,
          -1.9104e-01,  1.6555e-01, -6.4054e-02, -2.7901e-02,  4.0471e-02,
          -2.7128e-02,  1.2581e-02,  4.7945e-02,  3.1541e-02, -1.0502e-01]],

        [[-1.4861e-01, -9.9690e-03,  8.3975e-03, -2.2709e-01,  1.1541e-01,
          -6.1982e-02, -1.2137e-01, -2.8383e-02, -2.1029e-01,  5.4979e-02,
           1.6491e-01, -1.2694e-01, -2.4081e-01, -1.2147e-01, -1.0094e-01,
          -3.4948e-02,